In [1]:
import os, random, re, subprocess, sys, time, json
from pathlib import Path


In [17]:
# --- Cell 0: Notes & quick settings ---
# Recommended: run on Google Colab with GPU (Runtime -> Change runtime type -> GPU).
# Upload your SMSSpamCollection.csv into the Colab files panel or mount Drive and set DATA_PATH accordingly.

DATA_PATH = "/content/SMSSpamCollection.csv"   # change if you uploaded somewhere else
MODEL_NAME = "distilbert-base-uncased"
MAX_SAMPLES_PER_CLASS = 300      # keep dataset small => faster training
TEST_SIZE = 0.20
RANDOM_SEED = 42
BATCH_SIZE = 16
EPOCHS = 1                      # retrain quickly; raise to 2-3 if you want more
LR = 2e-5
MAX_SEQ_LENGTH = 128

# Attack settings (keep small for speed)
NUM_ATTACK_EXAMPLES = 100       # how many spam examples to attack (subset)
ATTACK_QUERY_BUDGET = 1000      # per-example query budget (TextAttack param)
ADD_ADV_TO_TRAIN_RATIO = 0.5    # fraction of successful adversarials to add to training for adversarial training

# ----------------------------------------------------------------------------


In [3]:
random.seed(RANDOM_SEED)

In [4]:
def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

try:
    import transformers, datasets, evaluate, torch, sklearn
except Exception:
    print("Installing required packages...")
    pip_install("transformers>=4.30.0")
    pip_install("datasets")
    pip_install("evaluate")
    pip_install("accelerate")
    pip_install("scikit-learn")
    # torch: choose the correct one for your CUDA version if you need GPU
    pip_install("torch")
    import transformers, datasets, evaluate, torch, sklearn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Installing required packages...
Device: cuda


In [5]:
def char_swap(text, prob=0.12):
    words = text.split()
    new_words = []
    for w in words:
        if len(w) > 2 and random.random() < prob:
            i = random.randint(0, len(w)-2)
            w = w[:i] + w[i+1] + w[i] + w[i+2:]
        new_words.append(w)
    return " ".join(new_words)

def add_ham_tokens(text, tokens=None, prob=0.6):
    if tokens is None:
        tokens = ["ok", "yes", "thanks", "lol", "okay", "sure"]
    if random.random() < prob:
        if random.random() < 0.6:
            return text + " " + random.choice(tokens)
        else:
            parts = text.split()
            pos = random.randint(0, len(parts))
            parts.insert(pos, random.choice(tokens))
            return " ".join(parts)
    return text

def generate_adversarial_variant(text):
    # Apply a single fast transform to keep generation lightweight
    if random.random() < 0.5:
        return char_swap(text)
    else:
        return add_ham_tokens(text)

In [6]:
df = pd.read_csv(DATA_PATH, header=None, encoding='latin-1')
if df.shape[1] >= 2:
    df = df.iloc[:, :2]
    df.columns = ["label", "message"]
else:
    raise ValueError("CSV must have at least two columns: label and message.")

# Clean and dedupe
df['message'] = df['message'].astype(str).str.strip()
df['label'] = df['label'].astype(str).str.strip().str.lower()
df = df.drop_duplicates()
df = df[df['label'].isin(['ham','spam'])].copy()
df['label_id'] = df['label'].map({'ham':0, 'spam':1})

print("Total after dedupe:", len(df))
print(df['label'].value_counts())

# Undersample majority to be equal to minority but also bound by MAX_SAMPLES_PER_CLASS
min_count = min(df['label'].value_counts().min(), MAX_SAMPLES_PER_CLASS)
df_balanced = pd.concat([
    df[df['label']=='spam'].sample(n=min_count, random_state=RANDOM_SEED) if df['label'].value_counts()['spam'] >= min_count else df[df['label']=='spam'],
    df[df['label']=='ham'].sample(n=min_count, random_state=RANDOM_SEED) if df['label'].value_counts()['ham'] >= min_count else df[df['label']=='ham'],
], ignore_index=True).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

print("Balanced dataset:", df_balanced.shape)
print(df_balanced['label'].value_counts())

# Stratified split
train_df, test_df = train_test_split(df_balanced, test_size=TEST_SIZE, stratify=df_balanced['label_id'], random_state=RANDOM_SEED)
print("Train/Test sizes:", train_df.shape, test_df.shape)
print("Train counts:\n", train_df['label'].value_counts())
print("Test counts:\n", test_df['label'].value_counts())

Total after dedupe: 5158
label
ham     4516
spam     642
Name: count, dtype: int64
Balanced dataset: (1000, 3)
label
ham     500
spam    500
Name: count, dtype: int64
Train/Test sizes: (800, 3) (200, 3)
Train counts:
 label
spam    400
ham     400
Name: count, dtype: int64
Test counts:
 label
spam    100
ham     100
Name: count, dtype: int64


In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    return tokenizer(batch["message"], truncation=True, padding="max_length", max_length=MAX_SEQ_LENGTH)

hf_train = Dataset.from_pandas(train_df[['message','label_id']].rename(columns={'label_id':'label'}))
hf_test = Dataset.from_pandas(test_df[['message','label_id']].rename(columns={'label_id':'label'}))

hf_train = hf_train.map(lambda x: tokenize_batch(x), batched=True)
hf_test  = hf_test.map(lambda x: tokenize_batch(x), batched=True)

# drop original text columns to keep tensors small (datasets may have different column names)
for d in [hf_train, hf_test]:
    for col in ["message","__index_level_0__"]:
        if col in d.column_names:
            d = d.remove_columns([col])

hf_train.set_format(type="torch", columns=["input_ids","attention_mask","label"])
hf_test.set_format(type="torch", columns=["input_ids","attention_mask","label"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [8]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

training_args = TrainingArguments(
    output_dir=os.path.join(OUTPUT_DIR, "baseline"),
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,

    save_strategy="no",
    logging_strategy="epoch",
    seed=RANDOM_SEED,
    fp16=torch.cuda.is_available(),
    disable_tqdm=False,
    # disable external logging backends (W&B, tensorboard, etc.)
    report_to=["none"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_test,
    compute_metrics=compute_metrics,
)

start = time.time()
trainer.train()
baseline_time = time.time() - start

baseline_eval = trainer.evaluate(eval_dataset=hf_test)
baseline_acc = baseline_eval.get("eval_accuracy", None)
print(f"\nBaseline accuracy on clean test: {baseline_acc:.4f} (train time {baseline_time:.1f}s)")

# classification report (clean)
preds_output = trainer.predict(hf_test)
preds = np.argmax(preds_output.predictions, axis=-1)
labels = preds_output.label_ids
print("\nClassification report (clean):\n", classification_report(labels, preds, target_names=["ham","spam"]))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.349900
100,0.077600



Baseline accuracy on clean test: 0.9850 (train time 8.2s)

Classification report (clean):
               precision    recall  f1-score   support

         ham       0.99      0.98      0.98       100
        spam       0.98      0.99      0.99       100

    accuracy                           0.98       200
   macro avg       0.99      0.98      0.98       200
weighted avg       0.99      0.98      0.98       200



In [12]:
# Generate adv_samples using numeric label column name 'label_id' (keeps things consistent)
spam_test_messages = test_df[test_df['label']=='spam']['message'].tolist()
adv_samples = []
for msg in spam_test_messages:
    for _ in range(N_ADVERSARIAL_PER_SPAM):
        adv = generate_adversarial_variant(msg)
        adv_samples.append({"message": adv, "label_id": 1})   # <- notice label_id

adv_samples = adv_samples[:MAX_ADVERSARIAL_SAMPLES]
print("Generated adv samples:", len(adv_samples))

# Build adv DataFrame with label_id and then append
adv_df = pd.DataFrame(adv_samples)  # columns: message, label_id

# Ensure test_df has message and label_id columns
# If test_df currently has 'label' (string) and 'label_id' (int), we will keep label_id
if 'label_id' not in test_df.columns:
    # if only 'label' exists (strings), map to label_id
    test_df['label_id'] = test_df['label'].map({'ham':0, 'spam':1})

adv_test_df = pd.concat([test_df.reset_index(drop=True), adv_df], ignore_index=True).reset_index(drop=True)

# Now create HF dataset using numeric 'label_id' -> rename to 'label' for HF Dataset
hf_adv_test = Dataset.from_pandas(adv_test_df[['message','label_id']].rename(columns={'label_id':'label'}))
hf_adv_test = hf_adv_test.map(lambda x: tokenizer(x["message"], truncation=True, padding="max_length", max_length=MAX_SEQ_LENGTH), batched=True)

# Drop text/index cols if present and set format
for col in ["message","__index_level_0__"]:
    if col in hf_adv_test.column_names:
        hf_adv_test = hf_adv_test.remove_columns([col])
hf_adv_test.set_format(type="torch", columns=["input_ids","attention_mask","label"])


adv_eval = trainer.evaluate(eval_dataset=hf_adv_test)
adv_acc = adv_eval.get("eval_accuracy", None)
print(f"\nAccuracy on attacked test (baseline): {adv_acc:.4f}")

preds_output_adv = trainer.predict(hf_adv_test)
preds_adv = np.argmax(preds_output_adv.predictions, axis=-1)
labels_adv = preds_output_adv.label_ids
print("\nClassification report (attacked):\n", classification_report(labels_adv, preds_adv, target_names=["ham","spam"]))

Generated adv samples: 100


Map:   0%|          | 0/300 [00:00<?, ? examples/s]


Accuracy on attacked test (baseline): 0.9867

Classification report (attacked):
               precision    recall  f1-score   support

         ham       0.98      0.98      0.98       100
        spam       0.99      0.99      0.99       200

    accuracy                           0.99       300
   macro avg       0.98      0.98      0.98       300
weighted avg       0.99      0.99      0.99       300



In [15]:
# --- Cell 6: Setup TextAttack and build attack (TextFooler) ---
# TextAttack imports
import textattack
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset as TA_Dataset

# Wrap HF model for TextAttack
# Important: ensure the model on device returns logits on CPU (TextAttack will call forward)
wrapped_model = HuggingFaceModelWrapper(model, tokenizer)

# Build TextFooler attack
attack = TextFoolerJin2019.build(wrapped_model)


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape s

In [14]:
pip_install("textattack")

In [18]:
# --- Cell 7: Prepare the dataset to attack (spam subset from test set) ---
# Extract spam examples from test set and limit to NUM_ATTACK_EXAMPLES
spam_test = test_df[test_df['label']=='spam'].reset_index(drop=True)
spam_to_attack = spam_test['message'].tolist()[:NUM_ATTACK_EXAMPLES]
labels_to_attack = [1]*len(spam_to_attack)    # label 1 == spam

# Create TextAttack dataset: list of (text, label) tuples
ta_dataset = TA_Dataset(list(zip(spam_to_attack, labels_to_attack)))
print("Prepared TA dataset length:", len(ta_dataset))


Prepared TA dataset length: 100


In [21]:
# --- Cell 8: Run attack (kept small) ---
attack_args = AttackArgs(
    num_examples=len(ta_dataset),   # attack all items in ta_dataset
    query_budget=ATTACK_QUERY_BUDGET,
    log_to_csv=None,
    checkpoint_interval=0,
    disable_stdout=False,           # show progress; set True to silence
    random_seed=RANDOM_SEED,
    parallel=False                  # set True for multi-process but more complex in Colab
)

attacker = Attacker(attack, ta_dataset, attack_args)
print("Starting attacks (this can take time depending on recipe and budget)...")
attack_results = attacker.attack_dataset()
print("Attacks complete.")

Starting attacks (this can take time depending on recipe and budget)...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModificati


  1%|          | 1/100 [00:31<52:25, 31.77s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/100 [00:31<52:25, 31.78s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (99%)]] --> [[[FAILED]]]

FreeMsg: Claim ur 250 SMS messages-Text OK to 84025 now!Use web2mobile 2 ur mates etc. Join Txt250.com for 1.50p/wk. T&C BOX139, LA32WU. 16 . Remove txtX or stop





[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   2%|▏         | 2/100 [00:33<27:44, 16.98s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   2%|▏         | 2/100 [00:33<27:44, 16.99s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (99%)]] --> [[[FAILED]]]

Urgent! Please call 09061213237 from landline. Â£5000 cash or a luxury 4* Canary Islands Holiday await collection. T&Cs SAE PO Box 177. M227XY. 150ppm. 16+




...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (99%)]] --> [[[FAILED]]]

If you don't, your prize will go to another customer. T&C at www.t-c.biz 18+ 150p/min Polo Ltd Suite 373 London W1J 6HL Please call back if busy





[Succeeded / Failed / Skipped / Total] 11 / 86 / 1 / 98:  99%|█████████▉| 99/100 [03:05<00:01,  1.87s/it]
[Succeeded / Failed / Skipped / Total] 11 / 87 / 1 / 99:  99%|█████████▉| 99/100 [03:05<00:01,  1.87s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[1 (99%)]] --> [[[FAILED]]]

Free 1st week entry 2 TEXTPOD 4 a chance 2 win 40GB iPod or Â£250 cash every wk. Txt VPOD to 81303 Ts&Cs www.textpod.net custcare 08712405020.





[Succeeded / Failed / Skipped / Total] 11 / 88 / 1 / 100: 100%|██████████| 100/100 [03:06<00:00,  1.87s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[1 (99%)]] --> [[[FAILED]]]

Call Germany for only 1 pence per minute! Call from a fixed line via access number 0844 861 85 85. No prepayment. Direct access!



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 88     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 99.0%  |
| Accuracy under attack:        | 88.0%  |
| Attack success rate:          | 11.11% |
| Average perturbed word %:     | 34.21% |
| Average num. words per input: | 25.62  |
| Avg num queries:              | 295.77 |
+-------------------------------+--------+
Attacks complete.


In [20]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [23]:
# Robust parsing of attack_results (works across TextAttack versions)
adv_examples = []
success_count = 0

for result in attack_results:
    # original text (string)
    try:
        original = result.original_text()  # TextAttack API
    except Exception:
        # fallback: try attributes
        original = getattr(result, "text_input", None) or getattr(result, "original_text", None) or ""
    # perturbed text (string or None)
    try:
        perturbed = result.perturbed_text()
    except Exception:
        # some result objects expose perturbed_text as attribute or via perturbed_result
        perturbed = None
        if hasattr(result, "perturbed_result") and result.perturbed_result is not None:
            try:
                perturbed = result.perturbed_result.output_text if hasattr(result.perturbed_result, "output_text") else None
            except Exception:
                perturbed = None

    # final fallback: if perturbed is still None, try to fetch from str(result)
    if perturbed is None:
        perturbed = None

    # Determine success robustly:
    # - primary: TextAttack often sets a status or goal_status enum
    # - fallback: treat as success if perturbed exists and is different from original
    success = False
    if hasattr(result, "goal_status"):
        try:
            success = (result.goal_status == textattack.shared.AttackResultStatus.SUCCEEDED)
        except Exception:
            success = False
    elif hasattr(result, "status"):
        # some versions use `status`
        try:
            success = (result.status == textattack.shared.AttackResultStatus.SUCCEEDED)
        except Exception:
            success = False
    else:
        # fallback: check perturbed_text existence and difference
        if perturbed is not None and isinstance(perturbed, str) and perturbed.strip() != "" and perturbed != original:
            success = True

    if success:
        success_count += 1

    # ground truth label: try to pick from result.original_result (if available) otherwise use 1 (spam)
    orig_label = None
    try:
        orig_label = result.original_result.ground_truth_output
    except Exception:
        orig_label = 1

    adv_examples.append({
        "original": original,
        "adv_text": perturbed if perturbed is not None else original,
        "success": bool(success),
        "orig_label": int(orig_label) if orig_label is not None else 1
    })

print(f"Total attacked: {len(adv_examples)}, successes: {success_count}")
adv_df = pd.DataFrame(adv_examples)
adv_df.head(8)


Total attacked: 100, successes: 99


,original,adv_text,success,orig_label
0,FreeMsg: Claim ur 250 SMS messages-Text OK to ...,FreeMsg: Argument ur 250 SMS messages-Text UM ...,True,1
1,Urgent! Please call 09061213237 from landline....,Expedited! Demanded invited 09061213237 from f...,True,1
2,For ur chance to win Â£250 cash every wk TXT: ...,Among gud occasion to winnings Â£250 funding e...,True,1
3,"Hi this is Amy, we will be sending you a free ...","Hi this is Amy, we will be sending you a free ...",False,1
4,FREE2DAY sexy St George's Day pic of Jordan!Tx...,FREE2DAY caliente St George's Jours pic of Ham...,True,1
5,FREE RINGTONE text FIRST to 87131 for a poly o...,FREE RINGTONE versions STARTED to 87131 for a ...,True,1
6,Do you want a New Nokia 3510i colour phone Del...,Ai you volition a Newest Nokia 3510i flush hea...,True,1
7,Todays Voda numbers ending 5226 are selected t...,Todays Voda numbering terminating 5226 are dec...,True,1


In [24]:
# --- Cell 10: Build adversarial test set and evaluate baseline model on it ---
# We'll append all perturbed texts (both successful and unsuccessful) to the attacked test set,
# or you can choose to include only successful adversarials (success==True). Here we include perturbed texts.
perturbed_texts = adv_df['adv_text'].tolist()
pert_labels = [1]*len(perturbed_texts)   # spam label

# Create a pandas adv test DF: we can append to original test_df or create fresh small adv test.
adv_test_small = pd.DataFrame({"message": perturbed_texts, "label": pert_labels})
# Tokenize and make HF dataset
hf_adv_test = Dataset.from_pandas(adv_test_small[['message','label']].rename(columns={'label':'label'}))
hf_adv_test = hf_adv_test.map(lambda x: tokenizer(x["message"], truncation=True, padding="max_length", max_length=MAX_SEQ_LENGTH), batched=True)
if "message" in hf_adv_test.column_names:
    hf_adv_test = hf_adv_test.remove_columns(["message"])
if "__index_level_0__" in hf_adv_test.column_names:
    hf_adv_test = hf_adv_test.remove_columns(["__index_level_0__"])
hf_adv_test.set_format(type="torch", columns=["input_ids","attention_mask","label"])

# Evaluate baseline model on adversarial examples
adv_eval = trainer.evaluate(eval_dataset=hf_adv_test)
adv_acc = adv_eval.get("eval_accuracy", None)
print(f"Baseline accuracy on TextAttack adversarials: {adv_acc:.4f}")

# For detail, get classification report by running predictions
preds_output_adv = trainer.predict(hf_adv_test)
preds_adv = np.argmax(preds_output_adv.predictions, axis=-1)
labels_adv = preds_output_adv.label_ids
print("\nClassification report on TextAttack adv set:\n", classification_report(labels_adv, preds_adv, target_names=["ham","spam"]))


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Baseline accuracy on TextAttack adversarials: 0.8800

Classification report on TextAttack adv set:
               precision    recall  f1-score   support

         ham       0.00      0.00      0.00         0
        spam       1.00      0.88      0.94       100

    accuracy                           0.88       100
   macro avg       0.50      0.44      0.47       100
weighted avg       1.00      0.88      0.94       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
# --- Cell 11: Save successful adversarials and optionally add to train set for adversarial training ---
successful_adv = adv_df[adv_df['success'] == True].reset_index(drop=True)
print("Successful adversarials count:", len(successful_adv))

# Decide how many to add to training set
n_to_add = int(len(successful_adv) * ADD_ADV_TO_TRAIN_RATIO)
adv_for_train = successful_adv['adv_text'].tolist()[:n_to_add]
print("Adding to train:", n_to_add)

# Build adv_train_df and append to train_df (we will map label 1)
if n_to_add > 0:
    adv_train_df = pd.DataFrame({"message": adv_for_train, "label_id": [1]*len(adv_for_train)})
    train_aug_df = pd.concat([train_df[['message','label_id']].rename(columns={'label_id':'label'}), adv_train_df.rename(columns={'label_id':'label'})], ignore_index=True).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
else:
    train_aug_df = train_df[['message','label_id']].rename(columns={'label_id':'label'})

# Create HF dataset for train_aug
hf_train_aug = Dataset.from_pandas(train_aug_df)
hf_train_aug = hf_train_aug.map(lambda x: tokenizer(x["message"], truncation=True, padding="max_length", max_length=MAX_SEQ_LENGTH), batched=True)
if "message" in hf_train_aug.column_names:
    hf_train_aug = hf_train_aug.remove_columns(["message"])
if "__index_level_0__" in hf_train_aug.column_names:
    hf_train_aug = hf_train_aug.remove_columns(["__index_level_0__"])
hf_train_aug.set_format(type="torch", columns=["input_ids","attention_mask","label"])

print("train_aug size:", len(hf_train_aug))


Successful adversarials count: 99
Adding to train: 49


Map:   0%|          | 0/849 [00:00<?, ? examples/s]

train_aug size: 849


In [27]:
# --- Cell 12: Retrain model with adversarial examples (adversarial training) ---
# Load fresh model (so we start from pretrained again) or continue training same model.
model_adv = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

training_args_adv = TrainingArguments(
    output_dir="adv_trained_run",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,

    save_strategy="no",
    logging_strategy="epoch",
    seed=RANDOM_SEED,
    report_to=["none"],
    fp16=torch.cuda.is_available(),
)

trainer_adv = Trainer(
    model=model_adv,
    args=training_args_adv,
    train_dataset=hf_train_aug,
    eval_dataset=hf_test,  # evaluate on clean test
    compute_metrics=compute_metrics,
)

start_adv = time.time()
trainer_adv.train()
adv_train_time = time.time() - start_adv

clean_eval_after = trainer_adv.evaluate(eval_dataset=hf_test)
clean_after_acc = clean_eval_after.get("eval_accuracy", None)
adv_eval_after = trainer_adv.evaluate(eval_dataset=hf_adv_test)
adv_after_acc = adv_eval_after.get("eval_accuracy", None)

print(f"Clean test accuracy after adversarial training: {clean_after_acc:.4f}")
print(f"Attacked test accuracy after adversarial training: {adv_after_acc:.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/100 [08:39<?, ?it/s]


Step,Training Loss
54,0.290400


Clean test accuracy after adversarial training: 0.9850
Attacked test accuracy after adversarial training: 0.9400


In [28]:
# --- Cell 13: Final reporting and saving artifacts ---
print("Summary:")
print("Baseline (clean) acc:", baseline_acc)
print("Baseline on TextAttack adv set:", adv_acc)
print("After adv training - clean:", clean_after_acc, "attacked:", adv_after_acc)

# Save adversarial samples and summary
os.makedirs("textattack_artifacts", exist_ok=True)
adv_df.to_csv("textattack_artifacts/attacks_raw.csv", index=False)
successful_adv.to_csv("textattack_artifacts/attacks_successful.csv", index=False)
with open("textattack_artifacts/summary.json","w") as f:
    json.dump({
        "baseline_clean_acc": float(baseline_acc),
        "baseline_adv_acc": float(adv_acc),
        "after_adv_training_clean_acc": float(clean_after_acc),
        "after_adv_training_adv_acc": float(adv_after_acc),
        "n_attacked": len(adv_df),
        "n_successful": len(successful_adv),
        "n_added_to_train": n_to_add
    }, f, indent=2)

print("Artifacts saved to ./textattack_artifacts")


Summary:
Baseline (clean) acc: 0.985
Baseline on TextAttack adv set: 0.88
After adv training - clean: 0.985 attacked: 0.94
Artifacts saved to ./textattack_artifacts
